# A Case Study: The Effect of Gun Ownership on Gun-Homicide Rates


We consider the problem of estimating the effect of gun ownership on the homicide rate. For this purpose, we perform inference on $\beta$ in the following the partially linear model:
$$
Y_{j, t}=\beta D_{j,(t-1)}+g\left(X_{j, t}, \bar{X}_j, \bar{X}_t, X_{j, 0}, Y_{j, 0}, t\right)+\epsilon_{j, t}
$$
$Y_{j, t}$ is the log homicide rate in county $j$ at time $t. D_{j, t-1}$ is the log fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership $G_{j, t}$, which is not observed. $X_{j, t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. We use $\bar{X}_j$ to denote the within county average of $X_{j, t}$ and $\bar{X}_t$ to denote the within time period average of $X_{j, t} . X_{j, 0}$ and $Y_{j, 0}$ denote initial conditions in county $j$. We use $Z_{j, t}$ to denote the set of observed control variables $\left\{X_{j, t}, \bar{X}_j, \bar{X}_t, X_{j, 0}, Y_{j, 0}, t\right\}$, so that our model is

$$
 Y_{i,t} = \beta D_{i,(t-1)} + g(Z_{i,t}) + \epsilon_{i,t}.
$$

## Data

$Y_{j,t}$ is the log homicide rate in county $j$ at time $t$, $D_{j, t-1}$ is the log fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership,  and  $Z_{j,t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. Assuming the firearm suicide rate is a good proxy for gun ownership, the parameter $\beta$ is the effect of gun ownership on homicide rates, controlling for county-level demographic and economic characteristics.

The sample covers 195 large United States counties between the years 1980 through 1999, giving us 3900 observations.

In [2]:
# Import relevant packages
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
warnings.simplefilter('ignore')

np.random.seed(1234)

In [4]:
file = "/content/sample_data/main.dta"
data = pd.read_stata(file)
data.shape

(1937, 73)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1937 entries, 0 to 1936
Data columns (total 73 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   province           1937 non-null   object 
 1   lati               1937 non-null   float32
 2   near_dist          1937 non-null   float32
 3   pat_per15          1937 non-null   float32
 4   gdpp               1937 non-null   float32
 5   samplenorth        1937 non-null   float32
 6   ele                1937 non-null   float32
 7   maizev             1937 non-null   float32
 8   ricewheat          1937 non-null   float32
 9   interricevm        1937 non-null   float32
 10  interwheatvm       1937 non-null   float32
 11  d                  1937 non-null   int8   
 12  edu                1937 non-null   float32
 13  above65            1937 non-null   float64
 14  migrenkou          1937 non-null   float32
 15  lower15            1937 non-null   float64
 16  r_pca1             1569 

In [6]:
data['interricevm'].describe()

,interricevm
count,1937.000000
mean,0.041127
std,0.053162
min,0.000000
25%,0.000567
50%,0.020750
75%,0.059994
max,0.204782


In [13]:
# Set the threshold for rice suitability (e.g., median)
df = data.copy()
threshold_rice = df['interricevm'].median()

# Create the dummy variable: 1 for high suitability, 0 for low suitability
df['interricevm_dummy'] = (df['interricevm'] > threshold_rice).astype(int)

# Check the new dummy variable
print(df[['interricevm', 'interricevm_dummy']].head())


   interricevm  interricevm_dummy
0     0.077000                  1
1     0.153256                  1
2     0.019850                  0
3     0.164175                  1
4     0.082059                  1


# Estimation

## Baseline OLS Estimates

After preprocessing the data, as a baseline model, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls in the full data set.



Next we estimate with the baseline set of controls.

<!-- Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics, we next include time and space averages. -->

We can also run our regression with time and space averages as controls.

Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics, we now include all controls.

In [15]:
# Regression on all controls

# Define the control variables (sum of certain variables + other selected variables)
X1 = df[["samplenorth", "near_dist", "ele", "lati", "i_1900", "d", "t", "p"]]  # Fixed the missing comma

# Define the treatment variables (assuming these are already defined)
# If you haven't already defined `T` as treatment variables, you can add them like this:
T = df[['interricevm_dummy']]  # For example, assuming dummies are used for treatment variables

# Combine treatment and control variables
X = pd.concat([T, X1], axis=1)  # Concatenate treatment and control variables column-wise

# Add a constant (intercept) term to the combined treatment and control variables
X_with_constant = sm.add_constant(X)  # Adding the intercept term

# Define the outcome variable (patent per capita)
y = df['pat_per15']  # Assuming this is your dependent variable

# Fit the OLS regression model
lm0 = sm.OLS(y, X_with_constant).fit(cov_type='HC3')  # Fit the model with robust standard errors (HC3)

# Print the coefficients and robust standard errors
print("OLS Coefficients:")
print(lm0.params)

# Extract the covariance matrix and calculate standard errors
vc0 = lm0.cov_params()  # Covariance matrix of the model parameters

# Print the coefficients and standard errors
std_err = np.sqrt(vc0.loc[['interricevm_dummy']])
print("\nStandard Errors:")
for coef, se in zip(lm0.params, std_err):
    print(f"{coef}: {se}")


OLS Coefficients:
const               -0.001907
interricevm_dummy   -0.000703
samplenorth         -0.001019
near_dist           -0.000069
ele                  0.000050
lati                 0.000018
i_1900               0.000037
d                    0.000100
t                    0.000211
p                   -0.000311
dtype: float64

Standard Errors:
-0.001907068885168552: const
-0.0007028748430942484: interricevm_dummy
-0.0010186592816147596: samplenorth
-6.851197569991093e-05: near_dist
5.021685361754247e-05: ele
1.7671197569358403e-05: lati
3.660665642407836e-05: i_1900
0.0001003393155659479: d
0.00021089677973827318: t
-0.0003107348774941093: p


In [16]:
lm0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              pat_per15   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.138
Method:                 Least Squares   F-statistic:                     18.68
Date:                Mon, 02 Dec 2024   Prob (F-statistic):           3.72e-30
Time:                        06:02:52   Log-Likelihood:                 10112.
No. Observations:                1937   AIC:                        -2.020e+04
Df Residuals:                    1927   BIC:                        -2.015e+04
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0019      0.002     -1.095      0.274      -0.005       0.002
interricevm_dummy    -0.0007      0.000     -4.755      0.000      -0.001      -0.000
samplenorth          -0.0010      0.000     -7.520      0.000      -0.001      -0.001
near_dist         -6.851e-05   1.04e-05     -6.592      0.000   -8.89e-05   -4.81e-05
ele                5.022e-05      0.000      0.335      0.737      -0.000       0.000
lati               1.767e-05   7.74e-06      2.283      0.022     2.5e-06    3.28e-05
i_1900             3.661e-05    9.9e-06      3.699      0.000    1.72e-05     5.6e-05
d                     0.0001   9.57e-05      1.049      0.294   -8.72e-05       0.000
t                     0.0002      0.000      1.672      0.095   -3.63e-05       0.000
p                    -0.0003      0.001     -0.572      0.568      -0.001       0.001
==============================================================================
Omnibus:                     2125.860   Durbin-Watson:                   1.888
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           144709.028
Skew:                           5.571   Prob(JB):                         0.00
Kurtosis:                      43.851   Cond. No.                     3.06e+03
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 3.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

After controlling for a rich set of characteristics, the point estimate of gun ownership attenuates to 0.179.

***NB***: In the background, `statsmodels` is dropping variables based on collinearity diagnostics. These depend on system linear algebra routines and can lead to large differences in high-dimensional or other ill-conditioned settings when using otherwise identical code across languages and/or machines.

Now we turn to our double machine learning framework, employing linear and flexible estimation methods with cross-fitting.

## DML Estimates

We perform inference on $\beta$ in the following the partially linear model:
 $$
Y_{j}=\beta D_{j}+g(Z_{j})+\epsilon_{j}.
$$
In the first stage, using cross-fitting, we employ modern regression methods to build estimators $\hat \ell(Z_{j,t})$ and $\hat m(Z_{j,t})$, where
- $\ell(Z_{j,t}):=E(Y_{j,t}|Z_{j,t})$
- $m(Z_{j,t}):=E(D_{j,t}|Z_{j,t})$

Using these, we obtain the estimates of the residualized quantities
- $\tilde Y_{j,t} = Y_{j,t}- E(Y_{j,t}|Z_{j,t})$
- $\tilde D_{j,t}= D_{j,t}- E(D_{j,t}|Z_{j,t})$

Using these residualized quantities, we note our model can be written as
$$
\tilde Y_{j,t} = \beta \tilde D_{j,t} + \epsilon_{j,t}, \quad E (\epsilon_{j,t} |\tilde D_{j,t}) =0.
$$
In the final stage, using ordinary least squares of $\tilde Y_{j,t}$ on $\tilde D_{j,t}$, we obtain the
estimate of $\beta$.

In the following, we consider 10 different methods for the first-stage models for $\ell(\cdot)$ and $m(\cdot)$ covering linear, penalized linear, and flexible methods. We also report the first-stage RMSE scores for estimating $Y$ and $D$.

In [17]:
def dml(X, D, y, modely, modeld, *, nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    time: array of time indices, eg [0,1,...,T-1,0,1,...,T-1,...,0,1,...,T-1]
    clu: array of cluster indices, eg [1073, 1073, 1073, ..., 5055, 5055, 5055, 5055]
    cluster: bool, whether to use clustered standard errors

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)  # shuffled k-folds
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1)  # out-of-fold predictions for y
    # out-of-fold predictions for D
    # use predict or predict_proba dependent on classifier or regressor for D
    if classifier:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    else:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    # calculate outcome and treatment residuals
    resy = y - yhat
    resD = D - Dhat

    dml_data = pd.concat([pd.Series(resy, name='resy'), pd.Series(resD, name='resD')], axis=1)
    ols_mod = smf.ols(formula='resy ~ 1 + resD', data=dml_data).fit()

    point = ols_mod.params[1]
    stderr = ols_mod.bse[1]
    epsilon = ols_mod.resid

    return point, stderr, yhat, Dhat, resy, resD, epsilon

In [19]:
def summary(point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y, *, name):
    '''
    Convenience summary function that takes the results of the DML function
    and summarizes several estimation quantities and performance metrics.
    '''
    return pd.DataFrame({'estimate': point,  # point estimate
                         'stderr': stderr,  # standard error
                         'lower': point - 1.96 * stderr,  # lower end of 95% confidence interval
                         'upper': point + 1.96 * stderr,  # upper end of 95% confidence interval
                         'rmse y': np.sqrt(np.mean(resy**2)),  # RMSE of model that predicts outcome y
                         'rmse D': np.sqrt(np.mean(resD**2))  # RMSE of model that predicts treatment D
                         }, index=[name])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1937 entries, 0 to 1936
Data columns (total 74 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   province           1937 non-null   object 
 1   lati               1937 non-null   float32
 2   near_dist          1937 non-null   float32
 3   pat_per15          1937 non-null   float32
 4   gdpp               1937 non-null   float32
 5   samplenorth        1937 non-null   float32
 6   ele                1937 non-null   float32
 7   maizev             1937 non-null   float32
 8   ricewheat          1937 non-null   float32
 9   interricevm        1937 non-null   float32
 10  interwheatvm       1937 non-null   float32
 11  d                  1937 non-null   int8   
 12  edu                1937 non-null   float32
 13  above65            1937 non-null   float64
 14  migrenkou          1937 non-null   float32
 15  lower15            1937 non-null   float64
 16  r_pca1             1569 

In [27]:
# OLS No Controls
Y = df['pat_per15']
D = df['interricevm_dummy']
Z = pd.DataFrame({"Const": np.ones(len(Y))})  # regression on constant

modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())

# Run DML model with nfolds folds of cross-fitting
result_OLS = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = summary(*result_OLS, Z, D, y, name='OLS No Controls')
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.00032,0.001413,0.500075


In [25]:
# Basic Controls
basic_controls = X1
Z = X1
modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())

# Run DML model with nfolds folds of cross-fitting
result_basic = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_basic, Z, D, y, name='OLS Basic Controls')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.000320,0.001413,0.500075
OLS Basic Controls,-0.000761,0.000092,-0.000940,-0.000581,0.001358,0.331180


In [28]:
# Basic Controls
basic_controls = X1
Z = sm.add_constant(X1)
modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())

# Run DML model with nfolds folds of cross-fitting
result_basic = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_basic, Z, D, y, name='OLS Basic Controls')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.000320,0.001413,0.500075
OLS Basic Controls,-0.000761,0.000092,-0.000940,-0.000581,0.001358,0.331180


In [29]:
# Now lets do Cross-validated Lasso, Ridge, ENet
cv = KFold(n_splits=10, shuffle=True, random_state=123)  # shuffled k-folds

In [30]:
# Define LassoCV models with n_splits folds of cross-validation
modely = make_pipeline(StandardScaler(), LassoCV(cv=cv))
modeld = make_pipeline(StandardScaler(), LassoCV(cv=cv))

# Run DML model with nfolds folds of cross-fitting
result_LassoCV = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_LassoCV, Z, D, y, name='LassoCV')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.000320,0.001413,0.500075
OLS Basic Controls,-0.000761,0.000092,-0.000940,-0.000581,0.001358,0.331180
LassoCV,-0.000737,0.000091,-0.000915,-0.000559,0.001347,0.331120


In [31]:
# Define RidgeCV models with n_splits folds of cross-validation
modely = make_pipeline(StandardScaler(), RidgeCV(cv=cv))
modeld = make_pipeline(StandardScaler(), RidgeCV(cv=cv))

# Run DML model with nfolds folds of cross-fitting
result_RidgeCV = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_RidgeCV, Z, D, y, name='RidgeCV')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.000320,0.001413,0.500075
OLS Basic Controls,-0.000761,0.000092,-0.000940,-0.000581,0.001358,0.331180
LassoCV,-0.000737,0.000091,-0.000915,-0.000559,0.001347,0.331120
RidgeCV,-0.000748,0.000091,-0.000927,-0.000570,0.001347,0.331081


In [32]:
# Define ElasticNetCV models with n_splits folds of cross-validation
modely = make_pipeline(StandardScaler(), ElasticNetCV(l1_ratio=0.5, cv=cv))
modeld = make_pipeline(StandardScaler(), ElasticNetCV(l1_ratio=0.5, cv=cv))

# Run DML model with nfolds folds of cross-fitting
result_ENetCV = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_ENetCV, Z, D, y, name='ENetCV')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.000320,0.001413,0.500075
OLS Basic Controls,-0.000761,0.000092,-0.000940,-0.000581,0.001358,0.331180
LassoCV,-0.000737,0.000091,-0.000915,-0.000559,0.001347,0.331120
RidgeCV,-0.000748,0.000091,-0.000927,-0.000570,0.001347,0.331081
ENetCV,-0.000736,0.000091,-0.000915,-0.000558,0.001347,0.331108


In [33]:
# DML with Random Forests. RFs don't require scaling but we do it for consistency
modely = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))

# Run DML model with nfolds folds of cross-fitting (computationally intensive)
result_RF = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_RF, Z, D, y, name='RF')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.000320,0.001413,0.500075
OLS Basic Controls,-0.000761,0.000092,-0.000940,-0.000581,0.001358,0.331180
LassoCV,-0.000737,0.000091,-0.000915,-0.000559,0.001347,0.331120
RidgeCV,-0.000748,0.000091,-0.000927,-0.000570,0.001347,0.331081
ENetCV,-0.000736,0.000091,-0.000915,-0.000558,0.001347,0.331108
RF,-0.000497,0.000115,-0.000722,-0.000272,0.001233,0.243023


In [34]:
# DML with Boosted Trees
modely = make_pipeline(StandardScaler(), GradientBoostingRegressor(max_depth=4, n_iter_no_change=5))
modeld = make_pipeline(StandardScaler(), GradientBoostingRegressor(max_depth=4, n_iter_no_change=5))

# Run DML model with nfolds folds of cross-fitting (computationally intensive)
result_BT = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_BT, Z, D, y, name='Boosted Trees')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.000320,0.001413,0.500075
OLS Basic Controls,-0.000761,0.000092,-0.000940,-0.000581,0.001358,0.331180
LassoCV,-0.000737,0.000091,-0.000915,-0.000559,0.001347,0.331120
RidgeCV,-0.000748,0.000091,-0.000927,-0.000570,0.001347,0.331081
ENetCV,-0.000736,0.000091,-0.000915,-0.000558,0.001347,0.331108
RF,-0.000497,0.000115,-0.000722,-0.000272,0.001233,0.243023
Boosted Trees,-0.000525,0.000118,-0.000757,-0.000293,0.001320,0.252222


In [35]:
# DML with NNs
modely = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(50, 50, 50, 50),
                                    activation='relu',
                                    solver='adam',
                                    alpha=0.0001,
                                    batch_size=200,
                                    learning_rate='constant',
                                    learning_rate_init=0.001,
                                    max_iter=200,
                                    shuffle=True,
                                    random_state=None,
                                    tol=1e-4,
                                    verbose=False,
                                    warm_start=False,
                                    momentum=0.9,
                                    nesterovs_momentum=True,
                                    early_stopping=True,
                                    validation_fraction=0.2,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                    epsilon=1e-08,
                                    n_iter_no_change=10)
                       )
modeld = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(50, 50, 50, 50),
                                    activation='relu',
                                    solver='adam',
                                    alpha=0.0001,
                                    batch_size=200,
                                    learning_rate='constant',
                                    learning_rate_init=0.001,
                                    max_iter=200,
                                    shuffle=True,
                                    random_state=None,
                                    tol=1e-4,
                                    verbose=False,
                                    warm_start=False,
                                    momentum=0.9,
                                    nesterovs_momentum=True,
                                    early_stopping=True,
                                    validation_fraction=0.2,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                    epsilon=1e-08,
                                    n_iter_no_change=10)
                       )

# Run DML model with nfolds folds of cross-fitting
result_NN = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_NN, Z, D, y, name='NN (Early Stopping)')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.000320,0.001413,0.500075
OLS Basic Controls,-0.000761,0.000092,-0.000940,-0.000581,0.001358,0.331180
LassoCV,-0.000737,0.000091,-0.000915,-0.000559,0.001347,0.331120
RidgeCV,-0.000748,0.000091,-0.000927,-0.000570,0.001347,0.331081
ENetCV,-0.000736,0.000091,-0.000915,-0.000558,0.001347,0.331108
RF,-0.000497,0.000115,-0.000722,-0.000272,0.001233,0.243023
Boosted Trees,-0.000525,0.000118,-0.000757,-0.000293,0.001320,0.252222
NN (Early Stopping),-0.001001,0.000557,-0.002092,0.000090,0.006767,0.275953


In [36]:
rmses = table.iloc[:, -2:]
rmses

,rmse y,rmse D
OLS No Controls,0.001413,0.500075
OLS Basic Controls,0.001358,0.331180
LassoCV,0.001347,0.331120
RidgeCV,0.001347,0.331081
ENetCV,0.001347,0.331108
RF,0.001233,0.243023
Boosted Trees,0.001320,0.252222
NN (Early Stopping),0.006767,0.275953


In [37]:
print("Lowest RMSE y: ", rmses.iloc[:, 0].idxmin())
print("Lowest RMSE D: ", rmses.iloc[:, 1].idxmin())

Lowest RMSE y:  RF
Lowest RMSE D:  RF


In [43]:
# DML with best model, which is RF
modely = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))

# Run DML model with nfolds folds of cross-fitting (computationally intensive)
result_best = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_best, Z, D, y, name='Best')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.000194,0.000064,0.000069,0.000320,0.001413,0.500075
OLS Basic Controls,-0.000761,0.000092,-0.000940,-0.000581,0.001358,0.331180
LassoCV,-0.000737,0.000091,-0.000915,-0.000559,0.001347,0.331120
RidgeCV,-0.000748,0.000091,-0.000927,-0.000570,0.001347,0.331081
ENetCV,-0.000736,0.000091,-0.000915,-0.000558,0.001347,0.331108
RF,-0.000497,0.000115,-0.000722,-0.000272,0.001233,0.243023
Boosted Trees,-0.000525,0.000118,-0.000757,-0.000293,0.001320,0.252222
NN (Early Stopping),-0.001001,0.000557,-0.002092,0.000090,0.006767,0.275953
Best,-0.000497,0.000115,-0.000722,-0.000272,0.001233,0.243023
Best,-0.000497,0.000115,-0.000722,-0.000272,0.001233,0.243023


In [42]:
# Least Squares Model Average
yhat_all = pd.concat([
    pd.Series(result_OLS[2]),
    pd.Series(result_basic[2]),
    pd.Series(result_LassoCV[2]),
    pd.Series(result_RidgeCV[2]),
    pd.Series(result_ENetCV[2]),
    pd.Series(result_RF[2]),
    pd.Series(result_BT[2]),
    pd.Series(result_NN[2])
], axis=1)

Dhat_all = pd.concat([
    pd.Series(result_OLS[3]),
    pd.Series(result_basic[3]),
    pd.Series(result_LassoCV[3]),
    pd.Series(result_RidgeCV[3]),
    pd.Series(result_ENetCV[3]),
    pd.Series(result_RF[3]),
    pd.Series(result_BT[3]),
    pd.Series(result_NN[3])
], axis=1)

ma_y = sm.OLS(df['pat_per15'], yhat_all).fit()
ma_d = sm.OLS(df['interricevm_dummy'], Dhat_all).fit()

weights_y = ma_y.params
weights_d = ma_d.params

lm_k = sm.OLS(ma_y.resid, ma_d.resid).fit(cov_type='HC3')
lsma = pd.Series({"estimate": lm_k.params[0], "stderr": lm_k.bse[0]},
                 name="Least Squares Model Average").to_frame().T

final_table = table.iloc[:, :2]
final_table = pd.concat([final_table, lsma], axis=0)
final_table

,estimate,stderr
OLS No Controls,0.000194,0.000064
OLS Basic Controls,-0.000761,0.000092
LassoCV,-0.000737,0.000091
RidgeCV,-0.000748,0.000091
ENetCV,-0.000736,0.000091
RF,-0.000497,0.000115
Boosted Trees,-0.000525,0.000118
NN (Early Stopping),-0.001001,0.000557
Best,-0.000497,0.000115
Least Squares Model Average,-0.000450,0.000161


In [44]:
# Convert 'final_table' to LaTeX format
latex_code = final_table.to_latex(index=False, float_format="%.3f", caption="Final Table", label="tab:final_table")

# Print the LaTeX code
print(latex_code)

\begin{table}
\caption{Final Table}
\label{tab:final_table}
\begin{tabular}{rr}
\toprule
estimate & stderr \\
\midrule
0.000 & 0.000 \\
-0.001 & 0.000 \\
-0.001 & 0.000 \\
-0.001 & 0.000 \\
-0.001 & 0.000 \\
-0.000 & 0.000 \\
-0.001 & 0.000 \\
-0.001 & 0.001 \\
-0.000 & 0.000 \\
-0.000 & 0.000 \\
\bottomrule
\end{tabular}
\end{table}

